In [1]:
import csv
import numpy as np
import pandas as pd
import sys, os
import re
import ast
import datetime as dt
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
diag = pd.read_csv('../data/hos/diagnoses_icd.csv').reset_index(drop = True)
# diag = diag.reset_index(inplace=True)
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4756321,19999987,23865745,7,41401,9
4756322,19999987,23865745,8,78039,9
4756323,19999987,23865745,9,0413,9
4756324,19999987,23865745,10,36846,9


In [3]:
diag['icd_version'].value_counts()
# diag.shape #(4756326, 6)

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [4]:
diag['new'] = diag['icd_code']

In [5]:
#adding underscore with the ICD version on the ICD_codes
diag.loc[diag["icd_version"] == 9, "new"] = '9_' + diag.loc[diag["icd_version"] == 9, "new"]
diag.loc[diag["icd_version"] == 10, "new"] = '10_' + diag.loc[diag["icd_version"] == 10, "new"]


In [6]:
diag.shape
diag.loc[diag["icd_version"] == 10]

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
40,10000084,23052089,1,G3183,10,10_G3183
41,10000084,23052089,2,F0280,10,10_F0280
42,10000084,23052089,3,R441,10,10_R441
43,10000084,23052089,4,R296,10,10_R296
44,10000084,23052089,5,E785,10,10_E785
...,...,...,...,...,...,...
4756291,19999828,29734428,18,Z9049,10,10_Z9049
4756292,19999828,29734428,19,Z87891,10,10_Z87891
4756293,19999828,29734428,20,B9620,10,10_B9620
4756294,19999828,29734428,21,Z1611,10,10_Z1611


In [7]:
d_diag = pd.read_csv('../data/hos/d_icd_diagnoses.csv').reset_index(drop = True)


# diag = diag.reset_index(inplace=True)
d_diag #109775 rows × 3 columns

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [8]:
d_diag.loc[d_diag['icd_version']==9]['icd_code'].nunique()#there are 14666 unique ICD code in version 9
d_diag.loc[d_diag['icd_version']==10]['icd_code'].nunique() #there are 95109 unique ICD code in version 10
diag.loc[diag['icd_version']==9]['icd_code'].nunique()#there are 9072 unique ICD code in version 9
diag.loc[diag['icd_version']==10]['icd_code'].nunique() #there are 16757 unique ICD code in version 10

16757

In [9]:
(diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'])
.reset_index())

,index,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title
0,0,10000032,22595853,1,5723,9,9_5723,Portal hypertension
1,1,10000826,20032235,4,5723,9,9_5723,Portal hypertension
2,2,10000826,28289260,1,5723,9,9_5723,Portal hypertension
3,3,10005866,26158160,4,5723,9,9_5723,Portal hypertension
4,4,10008924,23676183,7,5723,9,9_5723,Portal hypertension
...,...,...,...,...,...,...,...,...
4756321,4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini..."
4756322,4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ..."
4756323,4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,..."
4756324,4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...


In [10]:
diag_d = (diag
 .merge(d_diag, how='inner', left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version']))

In [24]:
print(diag_d.shape)
diag_d.head()

(4756326, 8)


,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572


In [12]:
d_diag

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A
...,...,...,...
109770,Z992,10,Dependence on renal dialysis
109771,Z993,10,Dependence on wheelchair
109772,Z998,10,Dependence on other enabling machines and devices
109773,Z9981,10,Dependence on supplemental oxygen


In [13]:
mappingicd9_icd10 = pd.read_csv("ICD9_to_ICD10_mapping.txt", delimiter='\t')

In [64]:
mappingicd9_icd10

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000
...,...,...,...,...,...,...
24280,ICD9,819,"MULTIPLE FRACTURES INVOLVING BOTH UPPER LIMBS,...",8190,S5290XA,10121
24281,ICD9,E888.0,FALL RESULTING IN STRIKING AGAINST SHARP OBJECT,E8880,W01110A,10000
24282,ICD9,V48.4,SENSORY PROBLEM WITH HEAD,V484,R6889,10000
24283,ICD9,802.25,ANGLE OF JAW CLOSED FRACTURE,80225,S02650A,10000


In [80]:
mappingicd9_icd10["icd9cm"].nunique() #12538

12538

In [14]:
mappingicd9_icd10.columns
mappingicd9_icd10.loc[mappingicd9_icd10['icd9cm']== '5723']

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
5589,ICD9,572.3,PORTAL HYPERTENSION,5723,K766,0


In [31]:
diag_d.loc[diag_d['icd_version']==9].shape #(2766877, 8) 5726301 rows × 10 columns
diag_d.loc[diag_d['icd_version']==10].shape #(1989449, 8)
diag_d['icd_version'].isin([9, 10]).shape #4756326

(4756326,)

In [47]:
2766877 - 5726301
# 4756326-5726301

-2959424

In [56]:
df9 = diag_d.loc[diag_d['icd_version']==9]
df9['icd_code'].value_counts()

4019     97361
2724     63875
53081    46238
25000    41029
42731    35180
         ...  
36119        1
37245        1
24971        1
3020         1
E8736        1
Name: icd_code, Length: 9072, dtype: int64

In [52]:
df9.shape #(2766877, 8))
diag_d['icd_version'].value_counts()

9     2766877
10    1989449
Name: icd_version, dtype: int64

In [54]:
diag_d['icd_version'].value_counts()

5717150-2766877 #2950273 mapping 24285row

2950273

In [44]:
diag_d

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572
...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",T24
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",O30
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",H35
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,K80


In [63]:
mappingicd9_icd10.drop_duplicates()

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000
...,...,...,...,...,...,...
24280,ICD9,819,"MULTIPLE FRACTURES INVOLVING BOTH UPPER LIMBS,...",8190,S5290XA,10121
24281,ICD9,E888.0,FALL RESULTING IN STRIKING AGAINST SHARP OBJECT,E8880,W01110A,10000
24282,ICD9,V48.4,SENSORY PROBLEM WITH HEAD,V484,R6889,10000
24283,ICD9,802.25,ANGLE OF JAW CLOSED FRACTURE,80225,S02650A,10000


In [61]:
mappingicd9_icd10['icd9cm'].value_counts()

V5412    533
V5416    528
V5411    213
V5413    159
99529    159
        ... 
73016      1
65973      1
76079      1
05472      1
V484       1
Name: icd9cm, Length: 12538, dtype: int64

In [76]:
mappingicd9_icd10.loc[mappingicd9_icd10['icd9cm']=='V5412'].drop_duplicates('icd9cm')

,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000


In [46]:
(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm']))

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572,5723,K766
...,...,...,...,...,...,...,...,...,...,...
5726296,19975121,28839110,3,0538,9,9_0538,Herpes zoster with unspecified complication,053,0538,B028
5726297,19983257,21588174,17,E9463,9,9_E9463,"Emollients, demulcents, and protectants causin...",E94,E9463,NoDx
5726298,19985683,25819866,6,2109,9,9_2109,"Benign neoplasm of pharynx, unspecified",210,2109,D109
5726299,19989305,29998115,2,74682,9,9_74682,Cor triatriatum,746,74682,Q242


In [78]:
(df9.merge(mappingicd9_icd10[['icd9cm', 'icd10cm']], 
           how='inner', left_on=['icd_code'], right_on=['icd9cm']))

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572,5723,K766
...,...,...,...,...,...,...,...,...,...,...
5717145,19975121,28839110,3,0538,9,9_0538,Herpes zoster with unspecified complication,053,0538,B028
5717146,19983257,21588174,17,E9463,9,9_E9463,"Emollients, demulcents, and protectants causin...",E94,E9463,NoDx
5717147,19985683,25819866,6,2109,9,9_2109,"Benign neoplasm of pharynx, unspecified",210,2109,D109
5717148,19989305,29998115,2,74682,9,9_74682,Cor triatriatum,746,74682,Q242


In [62]:
(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm'])).loc[(diag_d.merge
 (mappingicd9_icd10[['icd9cm', 'icd10cm']], 
  how='inner', left_on=['icd_code'], right_on=['icd9cm']))['icd_code']=='V5412']

#outer =7751133 rows

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3,icd9cm,icd10cm
5042519,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52602D
5042520,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52389D
5042521,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52559D
5042522,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52366D
5042523,10032409,25394829,14,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52355D
...,...,...,...,...,...,...,...,...,...,...
5060103,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52614D
5060104,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52212D
5060105,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S59041D
5060106,19523301,22310739,5,V5412,9,9_V5412,Aftercare for healing traumatic fracture of lo...,V54,V5412,S52221D


In [43]:
diag_d.loc[diag_d['icd_3']=='272']

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
642382,10000764,27897940,10,2724,9,9_2724,Other and unspecified hyperlipidemia,272
642383,10000935,21738619,10,2724,9,9_2724,Other and unspecified hyperlipidemia,272
642384,10000935,25849114,19,2724,9,9_2724,Other and unspecified hyperlipidemia,272
642385,10000935,26381316,16,2724,9,9_2724,Other and unspecified hyperlipidemia,272
642386,10000935,29541074,10,2724,9,9_2724,Other and unspecified hyperlipidemia,272
...,...,...,...,...,...,...,...,...
4720619,17268513,26840060,10,2729,9,9_2729,Unspecified disorder of lipoid metabolism,272
4737496,12271405,26160561,11,2723,9,9_2723,Hyperchylomicronemia,272
4737497,13203297,21559041,3,2723,9,9_2723,Hyperchylomicronemia,272
4737498,15518538,26699037,10,2723,9,9_2723,Hyperchylomicronemia,272


In [173]:
regex = r"[H/h]eart"

# regex = r"[K/k]idney"

regex = r"[D/d]igestive"

# test_str = diag_d['long_title']

# matches = re.findall(regex, test_str, re.MULTILINE)


# diag_d['long_title'].str.findall('[p/P]ortal')

diag_d[diag_d.long_title.str.match(regex)] #matcehs the beigning of the line

diag_d[diag_d.long_title.str.contains(regex)] #matches the pattern anywhere in the text

diag_d[diag_d.long_title.str.contains(regex)]['icd_code'].value_counts()

Z9049     2134
Z800      1506
99749     1311
9974      1293
K9189      674
          ... 
Q459         1
A1883        1
Z871         1
O99619       1
V127         1
Name: icd_code, Length: 64, dtype: int64

In [16]:
diag_d["icd_3"]=diag_d['icd_code'].astype(str).str[:3]
diag_d

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572
...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",T24
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",O30
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",H35
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,K80


In [17]:
diag_d['icd_3'].value_counts().to_frame().reset_index().rename(columns={'index':'icd_code3',
                                                                       'icd_3': 'counts'})

,icd_code3,counts
0,401,101252
1,272,83708
2,250,75310
3,V58,69592
4,E87,66157
...,...,...
2585,763,1
2586,N07,1
2587,V39,1
2588,N33,1


In [100]:
print(mappingicd9_icd10.shape)
mappingicd9_icd10.head()

(24285, 6)


,diagnosis_type,diagnosis_code,diagnosis_description,icd9cm,icd10cm,flags
0,ICD9,996.76,OTHER COMPLICATIONS DUE TO GENITOURINARY DEVIC...,99676,T8384XA,10000
1,ICD9,V54.12,AFTERCARE FOR HEALING TRAUMATIC FRACTURE OF LO...,V5412,S52602D,10000
2,ICD9,730.06,ACUTE OSTEOMYELITIS INVOLVING LOWER LEG,73006,M86169,10000
3,ICD9,345.61,"INFANTILE SPASMS, WITH INTRACTABLE EPILEPSY",34561,G40824,10000
4,ICD9,989.5,TOXIC EFFECT OF VENOM,9895,T63421A,10000


In [82]:
diag.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new
0,10000032,22595853,1,5723,9,9_5723
1,10000032,22595853,2,78959,9,9_78959
2,10000032,22595853,3,5715,9,9_5715
3,10000032,22595853,4,07070,9,9_07070
4,10000032,22595853,5,496,9,9_496


In [84]:
d_diag.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [93]:
diag_d.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572


In [102]:
icd9_icd10_dropduplicates = mappingicd9_icd10.drop_duplicates('icd9cm')

In [114]:
# df.drop(['B', 'C'], axis=1)
#diag_d.drop(columns=['icd_3']) #4756326 rows × 7 columns


((diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'inner',
       left_on= ['icd_code'],
       right_on=['icd9cm']).loc[(diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'inner',
       left_on= ['icd_code'],
       right_on=['icd9cm'])['icd_version']==10])


# (df9.merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], 
#            how = 'inner',
#            left_on= ['icd_code'],
#            right_on=['icd9cm']))  #2727971 rows
# (df9.merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], 
#            how = 'outer',
#            left_on= ['icd_code'],
#            right_on=['icd9cm'])) #2770706


,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd_3,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,572,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,572,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,572,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,572,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,572,5723,K766
...,...,...,...,...,...,...,...,...,...,...
2727966,19975121,28839110,3,0538,9,9_0538,Herpes zoster with unspecified complication,053,0538,B028
2727967,19983257,21588174,17,E9463,9,9_E9463,"Emollients, demulcents, and protectants causin...",E94,E9463,NoDx
2727968,19985683,25819866,6,2109,9,9_2109,"Benign neoplasm of pharynx, unspecified",210,2109,D109
2727969,19989305,29998115,2,74682,9,9_74682,Cor triatriatum,746,74682,Q242


In [118]:
2727971 - 2766877 #38906

2770706 - 2766877 #3829

2737057 - 4756326 #-2019269

-2019269

In [125]:
#merging the diag_d with the mapping ICD9 and icd10 files

print(diag_d.shape)
print(icd9_icd10_dropduplicates.shape)

# df['col1'] = df['col1'].fillna(df['col2'])

diag_d_icd = ((diag_d.drop(columns=['icd_3']))
 .merge(icd9_icd10_dropduplicates[['icd9cm','icd10cm']], how = 'left',
       left_on= ['icd_code'],
       right_on=['icd9cm']))

print(diag_d_icd.shape)
print(diag_d_icd.head())
print(diag_d_icd.tail())

(4756326, 8)
(12538, 6)
(4756326, 9)
   subject_id   hadm_id  seq_num icd_code  icd_version     new  \
0    10000032  22595853        1     5723            9  9_5723   
1    10000826  20032235        4     5723            9  9_5723   
2    10000826  28289260        1     5723            9  9_5723   
3    10005866  26158160        4     5723            9  9_5723   
4    10008924  23676183        7     5723            9  9_5723   

            long_title icd9cm icd10cm  
0  Portal hypertension   5723    K766  
1  Portal hypertension   5723    K766  
2  Portal hypertension   5723    K766  
3  Portal hypertension   5723    K766  
4  Portal hypertension   5723    K766  
         subject_id   hadm_id  seq_num icd_code  icd_version         new  \
4756321    19990427  29695607       24  T24011A           10  10_T24011A   
4756322    19996016  28015466        4   O30093           10   10_O30093   
4756323    19996783  25894657       20  H353131           10  10_H353131   
4756324    19997062  2

In [135]:
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,NaN
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,NaN
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,NaN
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,NaN


In [126]:
#there are nan in the df after merge how many

diag_d_icd.isna().sum()

subject_id           0
hadm_id              0
seq_num              0
icd_code             0
icd_version          0
new                  0
long_title           0
icd9cm         2019269
icd10cm        2019269
dtype: int64

In [133]:
#which rows are null

#for icd version 38906 rows are nan out of 2766877

icd9_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==9])[diag_d_icd.isnull().any(axis=1)]  

#for icd10 1980363 rows are nan.out of 1989449

icd10_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==10])[diag_d_icd.isnull().any(axis=1)]  

C:\Users\upadh\AppData\Local\Temp\ipykernel_12460\121918952.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  icd9_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==9])[diag_d_icd.isnull().any(axis=1)]
C:\Users\upadh\AppData\Local\Temp\ipykernel_12460\121918952.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  icd10_nan = (diag_d_icd.loc[diag_d_icd['icd_version']==10])[diag_d_icd.isnull().any(axis=1)]


In [137]:
diag_d_icd['icd10cm'] = diag_d_icd['icd10cm'].fillna(diag_d_icd['icd_code'])

In [138]:
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766
...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,T24011A
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,O30093
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,H353131
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,K8036


In [134]:
diag_d_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766


In [139]:
diag_d_icd["icd"]=diag_d_icd['icd10cm'].astype(str).str[:3]
diag_d_icd

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm,icd
0,10000032,22595853,1,5723,9,9_5723,Portal hypertension,5723,K766,K76
1,10000826,20032235,4,5723,9,9_5723,Portal hypertension,5723,K766,K76
2,10000826,28289260,1,5723,9,9_5723,Portal hypertension,5723,K766,K76
3,10005866,26158160,4,5723,9,9_5723,Portal hypertension,5723,K766,K76
4,10008924,23676183,7,5723,9,9_5723,Portal hypertension,5723,K766,K76
...,...,...,...,...,...,...,...,...,...,...
4756321,19990427,29695607,24,T24011A,10,10_T24011A,"Burn of unspecified degree of right thigh, ini...",NaN,T24011A,T24
4756322,19996016,28015466,4,O30093,10,10_O30093,"Twin pregnancy, unable to determine number of ...",NaN,O30093,O30
4756323,19996783,25894657,20,H353131,10,10_H353131,"Nonexudative age-related macular degeneration,...",NaN,H353131,H35
4756324,19997062,20096107,1,K8036,10,10_K8036,Calculus of bile duct with acute and chronic c...,NaN,K8036,K80


In [143]:
diag_d_icd['icd'].value_counts().to_frame().head(20)

,icd
E78,141010
E11,122226
Z79,120511
I25,109100
I16,98641
I50,86399
Z87,84852
K21,80446
E87,80201
F32,66252


In [151]:
diag_d_icd.loc[diag_d_icd['icd']=='I16']

,subject_id,hadm_id,seq_num,icd_code,icd_version,new,long_title,icd9cm,icd10cm,icd
441896,10000635,26134563,3,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441897,10000764,27897940,13,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441898,10001176,23334588,5,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441899,10001217,24597018,8,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
441900,10001217,27703517,6,4019,9,9_4019,Unspecified essential hypertension,4019,I169,I16
...,...,...,...,...,...,...,...,...,...,...
4538539,19265652,24073740,2,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538540,19476297,20894222,1,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538541,19480985,27592726,10,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16
4538542,19550378,27016663,1,I169,10,10_I169,"Hypertensive crisis, unspecified",NaN,I169,I16


In [172]:
diag_d_icd.loc[diag_d_icd['icd']=='K27']['long_title'].value_counts()

Peptic ulcer of unspecified site, unspecified as acute or chronic, without mention of hemorrhage or perforation, without mention of obstruction    843
Peptic ulcer, site unspecified, unspecified as acute or chronic, without hemorrhage or perforation                                                 341
Chronic peptic ulcer of unspecified site without mention of hemorrhage or perforation, without mention of obstruction                               51
Chronic or unspecified peptic ulcer of unspecified site with hemorrhage, without mention of obstruction                                             27
Chronic or unspecified peptic ulcer, site unspecified, with hemorrhage                                                                              14
Acute peptic ulcer of unspecified site with hemorrhage, without mention of obstruction                                                               3
Chronic peptic ulcer, site unspecified, without hemorrhage or perforation                     